# U-Net Architecture for single vertebra segmentation on crops received from YOLOv5



## Prepare data for model input

source data: dicom image contains full spine 
destination data: crop of single vertebrae (C3-C7, Th1-Th12, L1-L5, S1) ## C2 too different from others



In [ ]:
print("optimal image size: 4736 1920")
w = 256
h = 256
sizes = [(h, w), (h//2, w//2), (h//4, w//4), (h//8, w//8)]
print(sizes[0], sizes[1], sizes[2], sizes[3])

optimal image size: 4736 1920
(256, 256) (128, 128) (64, 64) (32, 32)


In [ ]:
import wandb
import albumentations
import torch
import os
from  functionality import *
import os
from sklearn.model_selection import train_test_split

c:\Users\gieko\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
path_to_dataset = "C:\\Users\\gieko\\Dropbox\\NIITO_Vertebrae\\NIITO_Vertebrae_Dataset\\NIITO_Vertebrae_Dataset_Final_resized\data_single_vertebra"

path_to_images = os.path.join(path_to_dataset, "images")
path_to_labels = os.path.join(path_to_dataset, "labels")

path_to_train = ""
path_to_val = ""
path_to_test = ""


all_cases = os.listdir(path_to_images)
# all_cases 

In [ ]:
width_max = 0
height_max = 0

width_min = 0
height_min = 0
shapes = []
for case in all_cases:
    current_path = os.path.join(path_to_labels, case)

    mask = read_mask(current_path)
    # print(list(mask.shape)[0:2] )
    # shapes.append(list(mask.shape[0]))
    xx, yy, zz = mask.nonzero()


    width = xx.max() - xx.min() + 16
    height = yy.max() - yy.min() + 16

    width_max = width if width > width_max else width_max
    height_max = height if height > height_max else height_max

    if case == '001_SD_C3.png':
        width_min = width
        height_min = height
    else:
        width_min = width if width < width_min else width_min
        height_min = height if height < height_min else height_min


    shapes.append(list(mask.shape)[0:2])

# print(shapes)

optimal_height, optimal_width = np.ceil(np.mean(np.array(shapes), axis = 0)).astype(int)
print("optimal image size:", optimal_width, optimal_height)
# optimal_height, optimal_width = optimal_height + (64 - optimal_height % 64), optimal_width + (128 - optimal_width % 128)
# print("optimal image size:", optimal_width, optimal_height)

optimal_height, optimal_width = optimal_height - optimal_height % 32, optimal_width - optimal_width % 32
print("max mask size: ", width_max, height_max)
print("min mask size: ", width_min, height_min)
print("min image size present", np.ceil(np.min(shapes, axis = 0)).astype(int))
print("max image size present", np.ceil(np.max(shapes, axis = 0)).astype(int))
print("optimal image size:", optimal_width, optimal_height)

optimal image size: 192 175
max mask size:  285 315
min mask size:  69 81
min image size present [83 95]
max image size present [299 328]
optimal image size: 192 160


# Dataset

In [ ]:
import os
import pydicom
from PIL import Image
from torch.utils.data import Dataset
import numpy as np

class DatasetNew(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return {"image": self.data[index]['image'], "mask": self.data[index]['mask']}

## Define augmentations

In [ ]:
import torch
# import torchvision 
import albumentations as A
from albumentations.pytorch import ToTensorV2
import albumentations.augmentations.functional as F


size = sizes[0]

quality_augs = [
            A.RandomFog(fog_coef_lower=0.3, fog_coef_upper=0.4, alpha_coef=0.05, p=0.125), 
            A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.2, p=0.125),
            A.RandomSnow(snow_point_lower=0.1, snow_point_upper=0.3, brightness_coeff=2.5, p=0.125),
            A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.125),
            A.CLAHE(p=0.125),
            A.GaussNoise(var_limit=(7.0, 27.0), mean=0, per_channel=False, p=0.125),
            A.Emboss(alpha=(0.2, 0.5), strength=(0.2, 0.7), p=0.125),
            A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.125),
        ]

rotation_augs = [
            A.Rotate(limit = (5,20), p=0.5),
            A.Rotate(limit = (340,355), p=0.5)
        ]

origs = A.ReplayCompose([
        A.OneOf([
            A.Resize(height=size[0], width=size[1]),
            A.Resize(height=size[0], width=size[1])], 
            p = 1),
        
        ToTensorV2()
    ], p=1,
    additional_targets={'image': 'image', 'mask': 'mask'})

origs_rotared = A.ReplayCompose([
        A.Resize(height=size[0], width=size[1]),
        A.OneOf(rotation_augs, p=1),
        ToTensorV2()
    ], p=1,
    additional_targets={'image': 'image', 'mask': 'mask'})

# one_effect = A.ReplayCompose([
#         A.Resize(height=size[0], width=size[1]),
#         A.OneOf(quality_augs, p=1),
#         ToTensorV2()
#     ], p=1,
#     additional_targets={'image': 'image', 'mask': 'mask'})

one_effect_rot = A.ReplayCompose([
        A.Resize(height=size[0], width=size[1]),
        A.OneOf(quality_augs, p=1),
        A.OneOf(rotation_augs, p=1),
        ToTensorV2()
    ], p=1,
    additional_targets={'image': 'image', 'mask': 'mask'})    

# two_effect = A.ReplayCompose([
#         A.Resize(height=size[0], width=size[1]),
#         A.OneOf(quality_augs, p=1),
#         A.OneOf(quality_augs, p=1),
#         ToTensorV2()
#     ], p=1,
#     additional_targets={'image': 'image', 'mask': 'mask'})

# two_effect_rot = A.ReplayCompose([
#         A.Resize(height=size[0], width=size[1]),
#         A.OneOf(quality_augs, p=1),
#         A.OneOf(quality_augs, p=1),
#         A.OneOf(rotation_augs, p=1),
#         ToTensorV2()
#     ], p=1,
#     additional_targets={'image': 'image', 'mask': 'mask'})

dataset_augs = [origs, origs_rotared, one_effect_rot]
# dataset_augs = [origs, origs_rotared, one_effect_rot, two_effect_rot]
# 


val_transforms = A.ReplayCompose(
    [   
        A.Resize(height=size[0], width=size[1]),
        ToTensorV2()
    ],
    additional_targets={'image': 'image', 'mask': 'mask'})

### Create DataLoader:One size images

In [ ]:
from sklearn.model_selection import train_test_split
train_cases, val_cases = train_test_split(all_cases, test_size=0.2)
print(len(train_cases), len(val_cases))


920 230


In [ ]:
from functionality import *


batch_size = 32
# kek_mask = 0
dataset = [[] for _ in range(len(dataset_augs))]
valid_aug = []
for case in train_cases:
    # print(case)
    path_mask = os.path.join(path_to_labels, case)
    path_image = os.path.join(path_to_images, case)

    image = cv2.imread(path_image, 1)
    # mask = read_mask(path_mask)
    mask = cv2.imread(path_mask, 0)   
    mask[mask==255] = 1.0 
    # print(np.array_equiv(mask, mask1))
    # print((mask > 0).sum(), (mask1 > 0).sum())
    # print(mask[mask > 0])
    # print(mask1[mask1 > 0])

    # plt.figure(figsize = (20,14))
    # plt.subplot(1,2,1)
    # plt.imshow(mask)
    # plt.subplot(1,2,2)
    # plt.imshow(mask1)

    # mask[mask==255.0] = 1.0
    # print()
    # print(image.shape, mask.shape)

    # plt.figure(figsize = (20,14))
    # plt.subplot(1, 2, 1)
    # plt.imshow(image)
    # plt.subplot(1, 2, 2)
    # plt.imshow(mask)


    for i, subset in enumerate(dataset_augs):
        augmentations = subset(image=image, mask=mask)
        # print(augmentations['image'].shape, augmentations['mask'].shape)
        dataset[i].append({
            "image": augmentations['image'],
            "mask": augmentations['mask']
        })
        # print(image.shape, mask.shape)

for case in val_cases:
    # print(case)
    path_mask = os.path.join(path_to_labels, case)
    path_image = os.path.join(path_to_images, case)

    image = cv2.imread(path_image, 1)
    # mask = read_mask(path_mask)
    mask = cv2.imread(path_mask, 0)
    # kek_mask = mask
    
    # image, mask, _ = read_image(path_image, path_mask, channels3=True)
    mask[mask==255.0] = 1.0

    # print(image.shape, mask.shape)
    # image = np.moveaxis(image, 0, 2)

    
    augmentations = val_transforms(image=image, mask=mask)
    print("!!",augmentations['image'].shape, augmentations['mask'].shape)

    valid_aug.append({
        "image": augmentations['image'],
        "mask": augmentations['mask']
    })


for subset in dataset:
    subset = DatasetNew(subset)

train_dataset = torch.utils.data.ConcatDataset(dataset)
train_loader =  torch.utils.data.DataLoader(train_dataset, batch_size, pin_memory=True, shuffle=True)

val_dataset = DatasetNew(valid_aug)
val_loader =  torch.utils.data.DataLoader(val_dataset, batch_size, pin_memory=True, shuffle=True)


# print(train_dataset, train_loader)
# print(train_dataset, train_loader)

# print(train_loader.__len__())
# print(val_loader.__len__())

print(train_dataset.__len__())
print(val_dataset.__len__())




!! torch.Size([3, 256, 256]) torch.Size([256, 256])
!! torch.Size([3, 256, 256]) torch.Size([256, 256])
!! torch.Size([3, 256, 256]) torch.Size([256, 256])
!! torch.Size([3, 256, 256]) torch.Size([256, 256])
!! torch.Size([3, 256, 256]) torch.Size([256, 256])
!! torch.Size([3, 256, 256]) torch.Size([256, 256])
!! torch.Size([3, 256, 256]) torch.Size([256, 256])
!! torch.Size([3, 256, 256]) torch.Size([256, 256])
!! torch.Size([3, 256, 256]) torch.Size([256, 256])
!! torch.Size([3, 256, 256]) torch.Size([256, 256])
!! torch.Size([3, 256, 256]) torch.Size([256, 256])
!! torch.Size([3, 256, 256]) torch.Size([256, 256])
!! torch.Size([3, 256, 256]) torch.Size([256, 256])
!! torch.Size([3, 256, 256]) torch.Size([256, 256])
!! torch.Size([3, 256, 256]) torch.Size([256, 256])
!! torch.Size([3, 256, 256]) torch.Size([256, 256])
!! torch.Size([3, 256, 256]) torch.Size([256, 256])
!! torch.Size([3, 256, 256]) torch.Size([256, 256])
!! torch.Size([3, 256, 256]) torch.Size([256, 256])
!! torch.Siz

# Utils

In [ ]:

import torch
import torch.nn as nn
import torchvision
import skimage.io
import numpy as np


def get_mean_std(loader):
    ch_sum, ch_squared_sum, count_of_batches = 0, 0, 0
    
    for data in loader:
        data = data['image'].float()
        data /= 255        

        ch_sum += torch.mean(data, dim=[0, 2, 3])
        ch_squared_sum += torch.mean(data**2, dim=[0, 2, 3])
        count_of_batches += 1

    mean = ch_sum / count_of_batches 
    std = (ch_squared_sum / count_of_batches - mean**2)**0.5

    return mean, std 


def soft_dice(*, y_true, y_pred):
    eps = 1e-15
    y_pred = y_pred.contiguous().view(y_pred.numel())
    y_true = y_true.contiguous().view(y_true.numel())
    intersection = (y_pred * y_true).sum(0)
    scores = 2. * (intersection + eps) / (y_pred.sum(0) + y_true.sum(0) + eps)
    score = scores.sum() / scores.numel()
    
    return torch.clamp(score, 0., 1.)


def hard_dice(*, y_true, y_pred, thr=0.5):
    y_pred = (y_pred > thr).float()
    return soft_dice(y_true=y_true, y_pred=y_pred)


def accuracy(y_true, y_pred, thr=0.5):
    num_correct = 0
    num_pixels = 0
    
    y_pred = (y_pred > thr).float()
    num_correct += (y_true == y_pred).sum()
    num_pixels += torch.numel(y_pred)
    
    return num_correct/num_pixels*100


class DiceLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, inputs, target):
        return 1 - soft_dice(y_true=target, y_pred=torch.sigmoid(inputs))


class BCEDiceLoss(nn.Module):
    def __init__(self, dice_weight=0.5):
        super().__init__()
        self._dice = DiceLoss()
        self._dice_weight = dice_weight

    def forward(self, inputs, target):
        return (1 - self._dice_weight) * nn.BCEWithLogitsLoss()(inputs, target) + \
            self._dice_weight * self._dice(inputs, target)




# Model

In [ ]:
import torch
import torch.nn as nn


class ConvLRelu(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, 3, padding=1, bias=False)
        self.batchNorm = nn.BatchNorm2d(out_channels)
        self.activation = nn.LeakyReLU(0.1)

    def forward(self, x):
        x = self.conv(x)
        x = self.batchNorm(x)
        x = self.activation(x)
        return x

    
class DoubleConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv_block = nn.Sequential(
            ConvLRelu(in_channels, out_channels),
            ConvLRelu(out_channels, out_channels),
        )
    
    def forward(self, x):
        x = self.conv_block(x)
        return x

class EncoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(EncoderBlock, self).__init__()
        self.conv_block = DoubleConvBlock(in_channels, out_channels)
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
            
    def forward(self, x):
        before_pool = self.conv_block(x)
        x = self.max_pool(before_pool)
        return x, before_pool
    
class DecoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DecoderBlock, self).__init__()              
        self.conv_block = DoubleConvBlock(in_channels, out_channels)

    def forward(self, x, y):
        x = nn.functional.interpolate(x, scale_factor=2, mode='bilinear', align_corners=True)
        return self.conv_block(torch.cat([x, y], dim=1))

class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, n_filters=64):
        super().__init__()
        self.pool = nn.MaxPool2d(2, 2)

        self.enc1 = EncoderBlock(in_channels, n_filters)
        self.enc2 = EncoderBlock(n_filters, n_filters * 2)
        self.enc3 = EncoderBlock(n_filters * 2, n_filters * 4)
        self.enc4 = EncoderBlock(n_filters * 4, n_filters * 8)
        
        self.center = DoubleConvBlock(n_filters * 8, n_filters * 16)
        # self.center = DoubleConvBlock(n_filters * 4, n_filters * 8)

        
        self.dec4 = DecoderBlock(n_filters * (16 + 8), n_filters * 8)
        self.dec3 = DecoderBlock(n_filters * (8 + 4), n_filters * 4)
        self.dec2 = DecoderBlock(n_filters * (4 + 2), n_filters * 2)
        self.dec1 = DecoderBlock(n_filters * (2 + 1), n_filters)

        self.final = nn.Conv2d(n_filters, out_channels, kernel_size=1)

    def forward(self, x):
        x = x.float()
        x, enc1 = self.enc1(x)
        x, enc2 = self.enc2(x)
        x, enc3 = self.enc3(x)
        x, enc4 = self.enc4(x)

        center = self.center(x)

        dec4 = self.dec4(center, enc4)
        dec3 = self.dec3(dec4, enc3)
        # dec3 = self.dec3(center, enc3)
        dec2 = self.dec2(dec3, enc2)
        dec1 = self.dec1(dec2, enc1)

        
        final = self.final(dec1)

        return final

# Trainer

In [ ]:
import tqdm
import os
from collections import defaultdict
import torch


class Trainer:
    CHECKPOINTS_PATH = 'checkpoints'
    
    def __init__(self, model, criterion, metric, optimizer, scheduler, config, device='cuda', start_epoch=0, loss=1.0, best_metric = float('-inf')):
        self._model = model
        self._optimizer = optimizer
        self._scheduler = scheduler
        self._criterion = criterion
        self._metrics = metric
        self._device = device
        

        self._epochs = config['epochs'] 
        self.start_epoch = start_epoch
        self.saved_loss = loss
        self._early_stopping = config['early_stopping']

        
        self._best_metric = best_metric
        if not os.path.exists(Trainer.CHECKPOINTS_PATH):
            os.makedirs(Trainer.CHECKPOINTS_PATH)

          
        
    
    def fit(self, train_loader, val_loader, save_to_drive=None):
        passed_epochs_without_upgrades = 0
        
        wandb.watch(self._model, self._criterion, log='all', log_freq=3)
        for epoch in range( self.start_epoch,  self.start_epoch + self._epochs):
            if passed_epochs_without_upgrades > self._early_stopping:
                return 
            
            self._model.train()
            train_metrics = self._run_epoch(epoch, train_loader, is_training=True)

            metrics_str = []
            for name, value in train_metrics.items():
                metrics_str.append(f'{name}: {float(value):.5f}')
            metrics_str = ' '.join(metrics_str)
            print('train metrics: ' + metrics_str)


            self._model.eval()
            val_metrics = self._run_epoch(epoch, val_loader, is_training=False)

            self._scheduler.step(val_metrics['dice'])
            print(self._optimizer.param_groups[0]['lr'])

            metrics_str = []
            for name, value in val_metrics.items():
                metrics_str.append(f'{name}: {float(value):.5f}')
            metrics_str = ' '.join(metrics_str)
            print('val metrics: ' + metrics_str)
            

            if self._best_metric < val_metrics['dice']:
                passed_epochs_without_upgrades = 0
                self._best_metric = val_metrics['dice']
                wandb.log({'saved at epoch': 1})
                torch.save(self._model.state_dict(), os.path.join(Trainer.CHECKPOINTS_PATH, 'weights.pth'))
                state = {
                    'epoch': epoch,
                    'state_dict': self._model.state_dict(),
                    'optimizer': self._optimizer.state_dict(),
                    'scheduler': self._scheduler.state_dict(),
                    'metrics': val_metrics
                }
                torch.save(state, os.path.join(Trainer.CHECKPOINTS_PATH, 'state.pth'))

                if save_to_drive is not None:
                  if wandb.run.name is not None:
                    if not os.path.exists(os.path.join(save_to_drive, wandb.run.name)):
                        path_to_folder_in_drive = os.path.join(save_to_drive, wandb.run.name)
                        os.makedirs(path_to_folder_in_drive)
                  else:
                    from datetime import datetime
                    name = datetime.today().strftime('%Y-%m-%d_%H-%M')
                    path_to_folder_in_drive = os.path.join(save_to_drive, name)
                    os.makedirs(path_to_folder_in_drive)
                  
                  torch.save(self._model.state_dict(), os.path.join(path_to_folder_in_drive, 'weights.pth'))
                  state = {
                      'epoch': epoch,
                      'state_dict': self._model.state_dict(),
                      'optimizer': self._optimizer.state_dict(),
                      'scheduler': self._scheduler.state_dict(),
                      'metrics': val_metrics
                  }
                  torch.save(state, os.path.join(path_to_folder_in_drive, 'state.pth'))
            else:
              wandb.log({'saved at epoch': 0})
           
            
            passed_epochs_without_upgrades += 1
    
    def _run_epoch(self, epoch, loader, is_training):
        if is_training:
            pbar = tqdm.tqdm(enumerate(loader), total=len(loader), desc=f'Epoch {epoch}')
        else:
            pbar = enumerate(loader)
        
        avg_metrics = defaultdict(float)
        index = np.random.randint(0, len(loader))
        for i, data in pbar:
            if i == index:
                images = data['image'].float().to(self._device)
                y_true = data['mask'].float().unsqueeze(1).to(self._device)
                y_pred = self._model(images)
                
            batch_metrics = self._step(data, is_training)
            for name, val in batch_metrics.items():
                avg_metrics[name] += val
        
        if not is_training:
            for name, val in avg_metrics.items():
                wandb.log({name: val / len(loader)})
            wandb.log({'learning_rate': self._optimizer.param_groups[0]['lr']})
            wandb.log({"sample": [wandb.Image(y_pred, caption=epoch)]})

        return {name: value / len(loader) for name, value in avg_metrics.items()}
    
    def _step(self, data, is_training=True):
        metrics_values = {}

        images = data['image'].float().to(self._device)
        y_true = data['mask'].float().unsqueeze(1).to(self._device)

        # print(images.shape, y_true.shape)
        
        if is_training:
            self._optimizer.zero_grad()

        with torch.set_grad_enabled(is_training):
            y_pred = self._model(images)
            loss = self._criterion(y_pred, y_true)
            
            for name, func in self._metrics:
                value = func(y_true=y_true, y_pred=torch.sigmoid(y_pred))
                metrics_values[name] = value.item()

            if is_training:
                loss.backward()
                self._optimizer.step()
        
        metrics_values['loss'] = loss.item()
        return metrics_values

# Training

In [ ]:
import wandb
wandb.login(key='f540abfade248550e8bc3abee9a5b1914d2ad798')



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: giekoolis. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\gieko/.netrc


True

In [ ]:
dir_path = os.getcwd()
save_path = os.path.join(dir_path, *["weight", "UNet_single-vertebrae"])
print(save_path)

# save_path = None

c:\Users\gieko\Dropbox\NIITO_Vertebrae\Scripts\weight\UNet_single-vertebrae


In [ ]:
os.environ["WANDB_MODE"]="offline" 

In [ ]:
device = "cpu"

hyperparametrs = {
    'n_filters': 32,
    'loss_weight': 0.8,
    'lr': 0.005,
    # 'lr': 1e-3,
    'epochs': 40,
    'lr_reduce_rate': 0.5,
    'patience': 3,
    'early_stopping': 50,
    'batch-size': batch_size,
    'model': 'UNet',
    'data': 'all cases',
    'image_size': size[0],
    'Description': 'Augmentations: only resize and rotate'
}

with wandb.init(project="NIITO-Single-Vertebrae-U-Net-Base", entity="giekoolis", config=hyperparametrs):
    config = hyperparametrs
    model = UNet(n_filters=config['n_filters'])
    criterion = BCEDiceLoss(config['loss_weight'])
    metric = [('accuracy', accuracy), ('dice', hard_dice)]
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            'max',
            factor=config['lr_reduce_rate'],
            patience=config['patience'],
            verbose=True
        )
    try:
        model.to(device)
        state = torch.load('C:\\Users\\gieko\\Dropbox\\NIITO_Vertebrae\\Scripts\\weight\\UNet_single-vertebrae\\jolly-wind-17\\state.pth', map_location=device)
        model.load_state_dict(state['state_dict'])
        optimizer.load_state_dict(state['optimizer'])
        scheduler.load_state_dict(state['scheduler'])
        # optimizer
        # optimizer.state_dict()["param_groups"][0]["lr"] = config['lr']
        
        
        # optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
        # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        #         optimizer,
        #         'max',
        #         factor=config['lr_reduce_rate'],
        #         patience=config['patience'],
        #         verbose=True
        #     )
        epoch = state['epoch']
        loss = state['metrics']['loss']
        dice = state['metrics']['dice']
        # print(optimizer.state_dict["lr"])
        trainer = Trainer(model, criterion, metric, optimizer, scheduler, config, device=device, start_epoch=epoch, loss=loss, best_metric=dice)
        print('load complete')
    except:
        trainer = Trainer(model, criterion, metric, optimizer, scheduler, config, device=device)
        print('load failed')
    trainer.fit(train_loader, val_loader, save_to_drive=save_path)



load complete


Epoch 27: 100%|██████████| 87/87 [34:25<00:00, 23.74s/it]


train metrics: accuracy: 95.38416 dice: 0.94802 loss: 0.07463
0.000625
val metrics: accuracy: 96.79739 dice: 0.96419 loss: 0.05161


Epoch 28: 100%|██████████| 87/87 [34:08<00:00, 23.55s/it]


train metrics: accuracy: 95.69028 dice: 0.95151 loss: 0.06922
0.000625
val metrics: accuracy: 96.44662 dice: 0.96084 loss: 0.05598


...

Epoch 46: 100%|██████████| 87/87 [37:36<00:00, 25.93s/it]


train metrics: accuracy: 97.34664 dice: 0.97014 loss: 0.04273
3.90625e-05
val metrics: accuracy: 96.51352 dice: 0.96114 loss: 0.05617


Epoch 47:   0%|          | 0/87 [00:07<?, ?it/s]


accuracy,█▃▅▃▁▆▅▅▄▄▂▄▄▅▅▅▄▅▅▄
dice,█▄▅▃▁▆▅▅▄▅▃▄▅▆▅▅▅▆▅▅
learning_rate,█████▄▄▄▄▂▂▂▂▁▁▁▁▁▁▁
loss,▁▅▃▆█▃▃▃▅▄▇▅▅▃▄▄▅▄▅▅
saved at epoch,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,96.51352
dice,0.96114
learning_rate,4e-05
loss,0.05617
saved at epoch,0


KeyboardInterrupt: 

In [ ]:
!wandb sync c:/Users/gieko/Dropbox/NIITO_Vertebrae/Scripts/wandb/offline-run-20230303_012852-vyx31okk

Find logs at: c:\Users\gieko\Dropbox\NIITO_Vertebrae\Scripts\wandb\debug-cli.gieko.log
Syncing: https://wandb.ai/giekoolis/NIITO-Single-Vertebrae-U-Net-Base/runs/vyx31okk ... done.


In [ ]:
kek = optimizer.state_dict()
print(optimizer.state_dict()["param_groups"][0]["lr"])
# print(optimizer.state_dict()["lr"])

7.8125e-06
